In [1]:
from learningAgents import ReinforceAlgorithm
from Environment import env, AdversaryModes
from NeuralNetwork import NeuralNetwork
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
from torch.distributions import Categorical
import time
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy

In [2]:
adversary_probabilities = torch.zeros(len(AdversaryModes))
adversary_probabilities[0]=1
env = env(total_demand = 400, tuple_costs = (57, 71), total_stages = 25, adversary_probabilities = adversary_probabilities)

/Users/edwardplumb/opt/anaconda3/lib/python3.9/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [3]:
model = PPO(MlpPolicy, env, verbose=0)

/Users/edwardplumb/opt/anaconda3/lib/python3.9/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [4]:
from stable_baselines3.common.base_class import BaseAlgorithm


def evaluate(
    model: BaseAlgorithm,
    num_episodes: int = 100,
    deterministic: bool = True,
) -> float:
    """
    Evaluate an RL agent for `num_episodes`.

    :param model: the RL Agent
    :param env: the gym Environment
    :param num_episodes: number of episodes to evaluate it
    :param deterministic: Whether to use deterministic or stochastic actions
    :return: Mean reward for the last `num_episodes`
    """
    # This function will only work for a single environment
    vec_env = model.get_env()
    obs = vec_env.reset()
    all_episode_rewards = []
    for _ in range(num_episodes):
        episode_rewards = []
        done = False
        # Note: SB3 VecEnv resets automatically:
        # https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecenv-api-vs-gym-api
        # obs = vec_env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            # `deterministic` is to use deterministic actions
            action, _states = model.predict(obs, deterministic=deterministic)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, _info = vec_env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print(f"Mean reward: {mean_episode_reward:.2f} - Num episodes: {num_episodes}")

    return mean_episode_reward

In [5]:
# Random Agent, before training
mean_reward_before_train = evaluate(model, num_episodes=1000, deterministic=True)

Mean reward: 162939.84 - Num episodes: 1000


In [6]:
model.learn(total_timesteps=500_000)

In [7]:
mean_reward_after_train = evaluate(model, num_episodes=1000, deterministic=True)

Mean reward: 175716.31 - Num episodes: 1000


In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, warn=False)

print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")